In [2]:
# Import necessary libraries
import pandas as pd              # For data manipulation and analysis
import plotly.express as px      # For interactive and expressive plots
import plotly.graph_objects as go  # For advanced plotly graphing capabilities
from dash import Dash, html, dcc  # For creating web-based dashboards
from dash.dependencies import Input, Output  # For defining callback functions


In [3]:
data = pd.read_csv("kiva_loans.csv")

data.head(5)

,id,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,partner_id,posted_time,disbursed_time,funded_time,term_in_months,lender_count,tags,borrower_genders,repayment_interval,date
0,653051,300.0,300.0,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,247.0,2014-01-01 06:12:39+00:00,2013-12-17 08:00:00+00:00,2014-01-02 10:06:32+00:00,12.0,12,NaN,female,irregular,2014-01-01
1,653053,575.0,575.0,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,PK,Pakistan,Lahore,PKR,247.0,2014-01-01 06:51:08+00:00,2013-12-17 08:00:00+00:00,2014-01-02 09:17:23+00:00,11.0,14,NaN,"female, female",irregular,2014-01-01
2,653068,150.0,150.0,Transportation,Transportation,To repair their old cycle-van and buy another ...,IN,India,Maynaguri,INR,334.0,2014-01-01 09:58:07+00:00,2013-12-17 08:00:00+00:00,2014-01-01 16:01:36+00:00,43.0,6,"user_favorite, user_favorite",female,bullet,2014-01-01
3,653063,200.0,200.0,Embroidery,Arts,to purchase an embroidery machine and a variet...,PK,Pakistan,Lahore,PKR,247.0,2014-01-01 08:03:11+00:00,2013-12-24 08:00:00+00:00,2014-01-01 13:00:00+00:00,11.0,8,NaN,female,irregular,2014-01-01
4,653084,400.0,400.0,Milk Sales,Food,to purchase one buffalo.,PK,Pakistan,Abdul Hakeem,PKR,245.0,2014-01-01 11:53:19+00:00,2013-12-17 08:00:00+00:00,2014-01-01 19:18:51+00:00,14.0,16,NaN,female,monthly,2014-01-01


## Kiva Loans

This file contains records from the Kiva Data Snapshot and can be matched to the loan theme regions to get a loan’s location.


### Column descriptions:

- **id**: Unique ID for loan
- **funded_amount**: The amount disbursed by Kiva to the field agent (USD)
- **loan_amount**: The amount disbursed by the field agent to the borrower (USD)
- **activity**: More granular category
- **sector**: High-level category
- **use**: Exact usage of loan amount
- **country_code**: ISO country code of the country in which the loan was disbursed
- **country**: Full country name of the country in which the loan was disbursed
- **region**: Full region name within the country
- **currency**: The currency in which the loan was disbursed
- **partner_id**: ID of the partner organization
- **posted_time**: The time at which the loan is posted on Kiva by the field agent
- **disbursed_time**: The time at which the loan is disbursed by the field agent to the borrower
- **funded_time**: The time at which the loan posted to Kiva gets funded by lenders completely
- **term_in_months**: The duration for which the loan was disbursed in months
- **lender_count**: The total number of lenders that contributed to this loan
- **tags**: Tags associated with the loan
- **borrower_genders**: Comma-separated M,F letters, where each instance represents a single male/female in the group
- **repayment_interval**: Interval for loan repayment
- **date**: Date of record



In [4]:
print("Dataset Overview:")
data.info()   # dataset overview

Dataset Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671205 entries, 0 to 671204
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  671205 non-null  int64  
 1   funded_amount       671205 non-null  float64
 2   loan_amount         671205 non-null  float64
 3   activity            671205 non-null  object 
 4   sector              671205 non-null  object 
 5   use                 666973 non-null  object 
 6   country_code        671197 non-null  object 
 7   country             671205 non-null  object 
 8   region              614405 non-null  object 
 9   currency            671205 non-null  object 
 10  partner_id          657698 non-null  float64
 11  posted_time         671205 non-null  object 
 12  disbursed_time      668809 non-null  object 
 13  funded_time         622874 non-null  object 
 14  term_in_months      671205 non-null  float64
 15  lender_count    

In [5]:
# Count the missing values in each column
missing_values = data.isnull().sum()

print("Missing Value\n ", missing_values)
print("----------------")
print("Dublicated Value : ",data.duplicated().sum()) # dublicate value

Missing Value
  id                         0
funded_amount              0
loan_amount                0
activity                   0
sector                     0
use                     4232
country_code               8
country                    0
region                 56800
currency                   0
partner_id             13507
posted_time                0
disbursed_time          2396
funded_time            48331
term_in_months             0
lender_count               0
tags                  171416
borrower_genders        4221
repayment_interval         0
date                       0
dtype: int64
----------------
Dublicated Value :  0


In [6]:

# Filter columns with missing values (count greater than zero)
columns_with_missing_values = missing_values[missing_values > 0]

if not columns_with_missing_values.empty:
    print("Columns with Missing Values:")
    print(columns_with_missing_values)
else:
    print("No missing values found in any column.")


Columns with Missing Values:
use                   4232
country_code             8
region               56800
partner_id           13507
disbursed_time        2396
funded_time          48331
tags                171416
borrower_genders      4221
dtype: int64


We've identified missing data in several columns: `['use', 'region', 'partner_id', 'disbursed_time', 'funded_time', 'tags', 'borrower_genders']`. While most of these may not significantly affect our analysis, it's worth noting that there are a few crucial columns — ` 'borrower_gender', 'disbursed_time', and 'funded_time'` — from which we can extract valuable insights.

In [7]:
print("\nSummary Statistics of Kiva Loans Dataset:")

data.describe().iloc[1:,[1,2,4,5]].style.background_gradient(cmap='viridis')



Summary Statistics of Kiva Loans Dataset:


,funded_amount,loan_amount,term_in_months,lender_count
mean,785.995061,842.397107,13.739022,20.590922
std,1130.398941,1198.660073,8.598919,28.459551
min,0.000000,25.000000,1.000000,0.000000
25%,250.000000,275.000000,8.000000,7.000000
50%,450.000000,500.000000,13.000000,13.000000
75%,900.000000,1000.000000,14.000000,24.000000
max,100000.000000,100000.000000,158.000000,2986.000000



### Key Performance Indicator: Loan Repayment Rate

The Loan Repayment Rate is a crucial Key Performance Indicator (KPI) in microfinance and lending industries. It measures the percentage of loan principal and interest that borrowers have repaid compared to the total amount borrowed. 

A high repayment rate indicates that borrowers are fulfilling their financial obligations, which reflects positively on the lender's portfolio performance and sustainability. Conversely, a low repayment rate may signal potential financial risks, such as defaults or delinquencies, which could impact the lender's profitability and ability to provide further loans.

Lenders use the repayment rate to assess the effectiveness of their lending strategies, the quality of borrowers, and the overall health of their loan portfolio. It helps them make informed decisions about future lending activities, risk management, and resource allocation.

Monitoring the repayment rate over time allows lenders to identify trends, patterns, and areas for improvement. They can implement targeted interventions, such as financial education programs or loan restructuring options, to support borrowers and enhance repayment behavior.

In summary, the Loan Repayment Rate is a critical metric that guides lenders in maintaining sustainable lending practices, fostering financial inclusion, and maximizing social impact.




In [8]:


# Count the number of IDs where loan amount equals funded amount
same_amount_count = len(data[data['loan_amount'] == data['funded_amount']])

print("Number of IDs with same loan amount and funded amount:", same_amount_count)



Number of IDs with same loan amount and funded amount: 622875


In [9]:
# Count the number of IDs where loan amount greater than funded amount
high_amount_count = len(data[data['loan_amount'] > data['funded_amount']])

print("Number of IDs with high loan amount and funded amount:", high_amount_count)



Number of IDs with high loan amount and funded amount: 48328


In [10]:
# Count the number of IDs where loan amount lesser than funded amount
low_amount_count = len(data[data['loan_amount'] < data['funded_amount']])

print("Number of IDs with low loan amount and funded amount:", low_amount_count)



Number of IDs with low loan amount and funded amount: 2


In [11]:
# Count the number of id where lfunded amount was 0 (yet to disbursed)
zero_amount_count = len(data[data.funded_amount < 1])

print("Number of IDs with zero funded amount:", zero_amount_count)


Number of IDs with zero funded amount: 3383


In [12]:
def is_paid(row):
    # Check if 'is_paid' column already has a value
    if 'is_paid' in row.index and pd.notnull(row['is_paid']):
        return row['is_paid']
    
    # Check if disbursed_time, funded_time, or funded_amount are NaN or 0
    if pd.isnull(row['disbursed_time']) or row['disbursed_time'] == 0 or \
       pd.isnull(row['funded_time']) or row['funded_time'] == 0 or \
       pd.isnull(row['funded_amount']) or row['funded_amount'] == 0:
        return False
    
    # Check if funded_amount equals loan_amount
    if row['funded_amount'] == row['loan_amount']:
        return True
    
    # Check based on repayment interval
    if row['repayment_interval'] in ['irregular', 'bullet']:
        return row['funded_amount'] == row['loan_amount']
    elif row['repayment_interval'] == 'monthly':
        return row['lender_count'] >= row['term_in_months']
    elif row['repayment_interval'] == 'weekly':
        return row['lender_count'] >= row['term_in_months'] * 4  # Assuming 4 weeks in a month
    
    return False  # Default case

# Apply the function to create a new column indicating whether the loan is paid
data['is_paid'] = data.apply(is_paid, axis=1)

# Display the updated dataframe
data.head()

,id,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,...,posted_time,disbursed_time,funded_time,term_in_months,lender_count,tags,borrower_genders,repayment_interval,date,is_paid
0,653051,300.0,300.0,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,...,2014-01-01 06:12:39+00:00,2013-12-17 08:00:00+00:00,2014-01-02 10:06:32+00:00,12.0,12,NaN,female,irregular,2014-01-01,True
1,653053,575.0,575.0,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,PK,Pakistan,Lahore,PKR,...,2014-01-01 06:51:08+00:00,2013-12-17 08:00:00+00:00,2014-01-02 09:17:23+00:00,11.0,14,NaN,"female, female",irregular,2014-01-01,True
2,653068,150.0,150.0,Transportation,Transportation,To repair their old cycle-van and buy another ...,IN,India,Maynaguri,INR,...,2014-01-01 09:58:07+00:00,2013-12-17 08:00:00+00:00,2014-01-01 16:01:36+00:00,43.0,6,"user_favorite, user_favorite",female,bullet,2014-01-01,True
3,653063,200.0,200.0,Embroidery,Arts,to purchase an embroidery machine and a variet...,PK,Pakistan,Lahore,PKR,...,2014-01-01 08:03:11+00:00,2013-12-24 08:00:00+00:00,2014-01-01 13:00:00+00:00,11.0,8,NaN,female,irregular,2014-01-01,True
4,653084,400.0,400.0,Milk Sales,Food,to purchase one buffalo.,PK,Pakistan,Abdul Hakeem,PKR,...,2014-01-01 11:53:19+00:00,2013-12-17 08:00:00+00:00,2014-01-01 19:18:51+00:00,14.0,16,NaN,female,monthly,2014-01-01,True


As we create new feature from exciting dataset, using some assumption, now we can easily calculate Loan Repayment Rate(LRR)
we created a new feature from the existing dataset and made some assumptions, and now  can easily calculate the Loan Repayment Rate (LRR) using the formula you provided:

LRR = (Total Amount Repaid / Total Amount Disbursed) * 100


In [13]:
# Assumpation 1
# Calculate the Loan Repayment Rate (LRR) using the new feature 
total_amount_repaid = data[data['is_paid']]['funded_amount'].sum()# Calculate total amount repaid based on your new feature
total_amount_disbursed = data['loan_amount'].sum() # Calculate total amount disbursed based on actual dataset

# Calculate the Loan Repayment Rate (LRR)
lrr = (total_amount_repaid / total_amount_disbursed) * 100

print("Loan Repayment Rate (LRR):", lrr)

Loan Repayment Rate (LRR): 87.91197057273149


In [14]:
# Assumpation 2 

# Calculate the mean of is_paid column to get the proportion of paid loans
proportion_paid = data['is_paid'].mean()

# Filter the DataFrame to get only the fully repaid loans
fully_repaid_loans = data[data['is_paid']]

# Calculate the total amount repaid for fully repaid loans
total_amount_repaid = fully_repaid_loans['funded_amount'].sum()

# Calculate the total amount disbursed for fully repaid loans
total_amount_disbursed = fully_repaid_loans['loan_amount'].sum()

# Calculate the Loan Repayment Rate (LRR) as a percentage
lrr = (total_amount_repaid / total_amount_disbursed) * 100

# Print the results
print("Proportion of paid loans:", proportion_paid)
print("Total amount repaid for fully repaid loans:", total_amount_repaid)
print("Total amount disbursed for fully repaid loans:", total_amount_disbursed)
print("Loan Repayment Rate (LRR):", lrr)


Proportion of paid loans: 0.9276346272748266
Total amount repaid for fully repaid loans: 497072875.0
Total amount disbursed for fully repaid loans: 497072875.0
Loan Repayment Rate (LRR): 100.0


In [15]:
# Calculate repayment rate by country
repayment_rate_by_country = data.groupby('country')['is_paid'].mean().reset_index()
repayment_rate_by_country = repayment_rate_by_country.sort_values(by='is_paid', ascending=False)

repayment_rate_by_country.head()


,country,is_paid
0,Afghanistan,1.0
67,Somalia,1.0
39,Lesotho,1.0
18,Cote D'Ivoire,1.0
44,Mauritania,1.0


In [16]:
top_10_countries = data.groupby('country')['is_paid'].mean().nlargest(10)
print(top_10_countries)

country
Afghanistan      1.0
Belize           1.0
Congo            1.0
Cote D'Ivoire    1.0
Israel           1.0
Lesotho          1.0
Mauritania       1.0
Namibia          1.0
Nepal            1.0
Panama           1.0
Name: is_paid, dtype: float64


In [17]:
small_10_countries = data.groupby('country')['is_paid'].mean().nsmallest(10)
print(small_10_countries)


country
Guam               0.000000
Virgin Islands     0.000000
Bhutan             0.500000
United States      0.634170
Puerto Rico        0.750000
Armenia            0.788553
Colombia           0.819868
El Salvador        0.820639
Myanmar (Burma)    0.836898
Azerbaijan         0.842159
Name: is_paid, dtype: float64


In [18]:
# Calculate repayment rate by sector
repayment_rate_by_sector = data.groupby('sector')['is_paid'].mean().reset_index()
repayment_rate_by_sector = repayment_rate_by_sector.sort_values(by='is_paid', ascending=False)
print(repayment_rate_by_sector)

            sector   is_paid
9    Manufacturing  0.992590
1             Arts  0.982338
4        Education  0.974914
14       Wholesale  0.966877
10    Personal Use  0.959819
3     Construction  0.946554
6             Food  0.940588
7           Health  0.932885
0      Agriculture  0.923001
11          Retail  0.917112
12        Services  0.913004
2         Clothing  0.908039
13  Transportation  0.888581
8          Housing  0.881266
5    Entertainment  0.873494


As there is multiple data in single row of borrower_gender column, it should be cleaned.

In [19]:
# Clean the 'borrower_genders' column
# Split the multiple genders into separate rows
data['borrower_genders'] = data['borrower_genders'].str.split(', ')

# Explode the 'borrower_genders' column to create separate rows for each gender
data = data.explode('borrower_genders')

# Now, 'borrower_genders' column will have one gender per row
# You may want to remove any leading or trailing whitespace
data['borrower_genders'] = data['borrower_genders'].str.strip()


In [20]:
# Calculate repayment rate by gender
repayment_rate_by_genders = data.groupby('borrower_genders')['is_paid'].mean().reset_index()
repayment_rate_by_genders = repayment_rate_by_genders.sort_values(by='is_paid', ascending=False)
print(repayment_rate_by_genders)

  borrower_genders   is_paid
0           female  0.951370
1             male  0.865746


In [21]:
# Sector Distribution
sector_distribution = data['sector'].value_counts(normalize=True)
print("Sector Distribution:")
print(sector_distribution)

Sector Distribution:
sector
Agriculture       0.278177
Food              0.241694
Retail            0.168669
Personal Use      0.075125
Clothing          0.065695
Services          0.056589
Housing           0.029968
Education         0.025992
Arts              0.017577
Transportation    0.013090
Health            0.011157
Construction      0.007797
Manufacturing     0.007023
Entertainment     0.000752
Wholesale         0.000696
Name: proportion, dtype: float64


In [22]:
# Geographic Distribution
geographic_distribution = data['country'].value_counts(normalize=True)
print("Geographic Distribution:")
print(geographic_distribution)

Geographic Distribution:
country
Kenya            1.211871e-01
Philippines      1.188071e-01
Paraguay         8.828428e-02
Cambodia         6.872388e-02
Peru             4.678499e-02
                     ...     
Afghanistan      1.481006e-06
Bhutan           1.481006e-06
Guam             7.405032e-07
Mauritania       7.405032e-07
Cote D'Ivoire    7.405032e-07
Name: proportion, Length: 87, dtype: float64


In [23]:
# Term in Months
term_distribution = data['term_in_months'].value_counts()
print("Term in Months Distribution:")
print(term_distribution)

Term in Months Distribution:
term_in_months
8.0      324359
14.0     210148
6.0      186387
11.0     115923
7.0       73023
          ...  
99.0          1
148.0         1
123.0         1
147.0         1
146.0         1
Name: count, Length: 148, dtype: int64


In [24]:
# Borrower Gender Distribution
gender_distribution = data['borrower_genders'].value_counts(normalize=True)
print("Borrower Gender Distribution:")
print(gender_distribution)


Borrower Gender Distribution:
borrower_genders
female    0.795794
male      0.204206
Name: proportion, dtype: float64


In [25]:
# Repayment Interval
repayment_interval_distribution = data['repayment_interval'].value_counts(normalize=True)
print("Repayment Interval Distribution:")
print(repayment_interval_distribution)

Repayment Interval Distribution:
repayment_interval
monthly      0.438148
irregular    0.397903
bullet       0.163503
weekly       0.000446
Name: proportion, dtype: float64


In [26]:
# Lender Count
lender_count = data.groupby('id')['lender_count'].max().mean()
print("Average Lender Count per Loan:", lender_count)

Average Lender Count per Loan: 20.590922296466804


In [27]:


# Initialize Dash app
app = Dash(__name__)

# Define layout
app.layout = html.Div([
    html.H1("Microfinance Loan Dashboard"),
    html.Label("Select visualization:"),
    dcc.Dropdown(
        id='visualization-dropdown',
        options=[
            {'label': 'Loan Amount by Country', 'value': 'country'},
            {'label': 'Loan Amount by Sector', 'value': 'sector'},
            {'label': 'Loan Amount by Gender', 'value': 'gender'},
            {'label': 'Distribution of Loan Amounts', 'value': 'distribution'}
        ],
        value='country'
    ),
    dcc.Graph(id='loan-visualization')
])

# Define callback to update graph based on dropdown selection
@app.callback(
    Output('loan-visualization', 'figure'),
    [Input('visualization-dropdown', 'value')]
)
def update_graph(selected_option):
    if selected_option == 'country':
        loan_amount_by_country = data.groupby('country').agg({'loan_amount': 'sum'}).reset_index()
        fig = go.Figure(go.Bar(x=loan_amount_by_country['country'], y=loan_amount_by_country['loan_amount'], 
                               marker_color='skyblue'))
        fig.update_layout(xaxis_title="Country", yaxis_title="Loan Amount", title="Loan Amount by Country")
        return fig
    elif selected_option == 'sector':
        loan_amount_by_sector = data.groupby('sector').agg({'loan_amount': 'sum'}).reset_index()
        fig = go.Figure(go.Bar(x=loan_amount_by_sector['sector'], y=loan_amount_by_sector['loan_amount'], 
                               marker_color='salmon'))
        fig.update_layout(xaxis_title="Sector", yaxis_title="Loan Amount", title="Loan Amount by Sector")
        return fig
    elif selected_option == 'gender':
        loan_amount_by_gender = data.groupby('borrower_genders').agg({'loan_amount': 'sum'}).reset_index()
        fig = go.Figure(go.Bar(x=loan_amount_by_gender['borrower_genders'], y=loan_amount_by_gender['loan_amount'], 
                               marker_color='lightgreen'))
        fig.update_layout(xaxis_title="Gender", yaxis_title="Loan Amount", title="Loan Amount by Gender")
        return fig
    elif selected_option == 'distribution':
        fig = go.Figure(go.Histogram(x=data['loan_amount'], marker_color='orange'))
        fig.update_layout(xaxis_title="Loan Amount", yaxis_title="Count", title="Distribution of Loan Amounts")
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, mode='inline', port=8060)
    
